# Italian-LEGAL-BERT

In [1]:
import os
import optuna
import pandas as pd
import sys
import torch
import transformers
import wandb

sys.path.insert(0, os.path.abspath('../'))

from rhetorical_roles_classification import (
    RhetoricalRolesDataset,
)
from rhetorical_roles_classification.train import train
from rhetorical_roles_classification.test import test_BERT

from config import (
    DATA_FOLDER,
    MODELS_FOLDER,
    HUGGING_FACE_BERT_MODEL,
    NUM_LABELS,
    WANDB_PROJECT,
    MAX_SEGMENT_LENGTH,
    label2rhetRole
)

In [2]:
MODEL_NAME = "Italian-LEGAL-BERT"

DEVICE = "cuda:0"

wandb_login_success = wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gmarinowandb-8601. Use `wandb login --relogin` to force relogin


## Dataset Loading

In [3]:
train_dataset = RhetoricalRolesDataset(
    data_filepath=os.path.join(DATA_FOLDER, "train.csv"),
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL,
    max_segment_length=MAX_SEGMENT_LENGTH
)
valid_dataset = RhetoricalRolesDataset(
    data_filepath=os.path.join(DATA_FOLDER, "valid.csv"),
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL,
    max_segment_length=MAX_SEGMENT_LENGTH
)
test_dataset = RhetoricalRolesDataset(
    data_filepath=os.path.join(DATA_FOLDER, "test.csv"),
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL,
    max_segment_length=MAX_SEGMENT_LENGTH
)

## Hyperparameter Tuning

In [4]:
hyperparameters = {
    "batch_size": 128,
    "epochs": 20,
    "num_warmup_steps": 0,
    "accum_iter": 3,
    "early_stopping": True,
    "early_stopping_patience": 2
}

def objective(trial):
    
    print("\n************************ Starting New Optuna Trial ************************\n")
    
    hyperparameters["lr"] = trial.suggest_float("lr", 5e-6, 5e-4)
    hyperparameters["weight_decay"] = trial.suggest_float("weight_decay", 1e-3, 1e-1)

    model = transformers.AutoModelForSequenceClassification.from_pretrained(
        HUGGING_FACE_BERT_MODEL,
        num_labels=NUM_LABELS,
    )
    model_id = f"{MODEL_NAME}-Trial{trial.number}"

    with wandb.init(project=WANDB_PROJECT, config=hyperparameters, name=model_id):

        valid_loss = train(
            model=model,
            destination_path=os.path.join(MODELS_FOLDER, model_id),
            train_dataset=train_dataset,
            valid_dataset=valid_dataset,
            device=DEVICE,
            optuna_trial=trial,
            use_wandb=True,
            **hyperparameters
        )
        
    return valid_loss

In [4]:
study_name = f"{MODEL_NAME}.optuna"
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="minimize")

[I 2023-04-29 12:22:55,723] Using an existing study with name 'Italian-LEGAL-BERT.optuna' instead of creating a new one.


In [6]:
study.optimize(objective, n_trials=32)


************************ Starting New Optuna Trial ************************



Some weights of the model checkpoint at dlicari/Italian-Legal-BERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dlicari/Italian-Le


************** Training Started **************

EPOCH N. 1


532it [01:47,  4.97it/s]



TRAIN RESULTS
Loss: 0.004335491501032826
Accuracy: 0.7971681467976239
MCC: 0.6778517139072799
Macro F1: 0.7766420878741671
Macro Precision: 0.8106312525570015
Macro Recall: 0.7522371829134281
Micro F1: 0.7971681467976239
Micro Precision: 0.7971681467976239
Micro Recall: 0.7971681467976239

VALIDATION RESULTS
Loss: 0.003135881619527936
Accuracy: 0.8497920997920998
MCC: 0.7686455327102741
Macro F1: 0.8595188196635604
Macro Precision: 0.8591071701069609
Macro Recall: 0.8603353232267926
Micro F1: 0.8497920997920998
Micro Precision: 0.8497920997920998
Micro Recall: 0.8497920997920998
--------------------------------------------------
EPOCH N. 2


...


TRAIN RESULTS
Loss: 0.0023968443855102543
Accuracy: 0.8844468623184144
MCC: 0.8193238010092605
Macro F1: 0.8873770425323293
Macro Precision: 0.8917933687606971
Macro Recall: 0.8840438438941158
Micro F1: 0.8844468623184144
Micro Precision: 0.8844468623184144
Micro Recall: 0.8844468623184144

VALIDATION RESULTS
Loss: 0.0030705060344189405
Accuracy: 0.8641372141372141
MCC: 0.7893346560936495
Macro F1: 0.8675287513061773
Macro Precision: 0.880583057989828
Macro Recall: 0.8590985203765061
Micro F1: 0.8641372141372141
Micro Precision: 0.8641372141372141
Micro Recall: 0.8641372141372141
--------------------------------------------------
EPOCH N. 3


532it [01:42,  5.19it/s]



TRAIN RESULTS
Loss: 0.001882438961069654
Accuracy: 0.9102364288654943
MCC: 0.8598906066449791
Macro F1: 0.9120798284664792
Macro Precision: 0.9152698234498953
Macro Recall: 0.9092503453034462
Micro F1: 0.9102364288654943
Micro Precision: 0.9102364288654943
Micro Recall: 0.9102364288654943

VALIDATION RESULTS
Loss: 0.0031825038604438305
Accuracy: 0.8627858627858628
MCC: 0.7892511504852846
Macro F1: 0.8683184263121368
Macro Precision: 0.8743826447230194
Macro Recall: 0.8685525204285323
Micro F1: 0.8627858627858628
Micro Precision: 0.8627858627858628
Micro Recall: 0.8627858627858628
Early stopped training
--------------------------------------------------


Epoch,▁▅█
Train Loss,█▃▁
Validation Loss,▁▆█
Epoch,3
Train Loss,0.00188
Validation Loss,0.00318


[I 2023-04-24 21:59:50,141] Trial 31 finished with value: 0.0028390465304255486 and parameters: {'lr': 0.00011409531137229585, 'weight_decay': 0.027704244707390878}. Best is trial 12 with value: 0.0027747247368097305.


In [5]:
print("\n************************ Optuna Study Completed ************************\n")
best_trial = study.best_trial
print("Best trial:", best_trial.number)
print("Value:", best_trial.value)
print("Params:")
for key, value in best_trial.params.items():
    print(f"\t{key}: {value}")


************************ Optuna Study Completed ************************

Best trial: 12
Value: 0.0027747247368097305
Params:
	lr: 6.49435749751354e-05
	weight_decay: 0.053536308903641604


## Test

In [6]:
best_trial = study.best_trial
best_params = best_trial.params

model = transformers.AutoModelForSequenceClassification.from_pretrained(
    HUGGING_FACE_BERT_MODEL,
    num_labels=NUM_LABELS,
)
model_id = f"{MODEL_NAME}-Trial{best_trial.number}"
model.load_state_dict(torch.load(os.path.join(MODELS_FOLDER, model_id)))

Some weights of the model checkpoint at dlicari/Italian-Legal-BERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dlicari/Italian-Le

<All keys matched successfully>

In [7]:
test_BERT(
    model=model,
    test_dataset=test_dataset,
    label2rhetRole=label2rhetRole,
    device=DEVICE
)

100%|██████████| 143/143 [00:18<00:00,  7.77it/s]


Overall
	Accuracy: 0.8720472440944882
	MCC: 0.8061252310760577
	Macro F1: 0.8781011379790327
	Macro Precision: 0.8717954154090514
	Macro Recall: 0.8885260605841208
	Micro F1: 0.8720472440944882
	Micro Precision: 0.8720472440944882
	Micro Recall: 0.8720472440944882
Rhetorical role: UNK
	Macro F1: 0.9895155349295848
	Macro Precision: 0.9849383905205003
	Macro Recall: 0.9941903132531236
Rhetorical role: CONCLUSIONI DELLE PARTI
	Macro F1: 0.8962120963954674
	Macro Precision: 0.8659435443574401
	Macro Recall: 0.9351504558781598
Rhetorical role: SVOLGIMENTO DEL PROCESSO
	Macro F1: 0.8489107320183222
	Macro Precision: 0.8608255299548514
	Macro Recall: 0.838681728114644
Rhetorical role: MOTIVI DELLA DECISIONE
	Macro F1: 0.9123647343233967
	Macro Precision: 0.9122665511231973
	Macro Recall: 0.9124970684311142
Rhetorical role: P.Q.M.
	Macro F1: 0.9512013069674099
	Macro Precision: 0.9574066270974255
	Macro Recall: 0.9451980656743505
